In [1]:
# Packages used in this notebook
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import configparser

# setting the starting url 
url_overview_köln = "https://www.wg-gesucht.de/wg-zimmer-in-Koeln.73.0.0.1.html"

## First testing

In [2]:
# Testing 
# Sends a GET request to the URL to retrieve its content; The Website tryes to redicrect the request to a captcha page, reloading solves the problem
response = requests.get(url_overview_köln, allow_redirects=False)
# assert response.status_code == 200
print(response.status_code)
# Parse results
soup = BeautifulSoup(response.text, 'html.parser')

target_class = "ang_spalte_icons row_click"
elements = soup.find_all('td', class_=target_class)
links=[]
for element in elements:
    links.append(element.find('a').get('href'))
# Extracting all shared apartments
#for link in soup.find_all('a'):
#    print(link.get('href'))
print(links)
print(len(links))


200
['wg-zimmer-in-Koeln.10965760.html', 'wg-zimmer-in-Koeln-Nippes.11036496.html', 'wg-zimmer-in-Koeln-Nippes.11072804.html', 'wg-zimmer-in-Koeln-Humboldt-Gremberg.11072801.html', 'wg-zimmer-in-Koeln-Weiden.11013117.html', 'wg-zimmer-in-Koeln.11020451.html', 'wg-zimmer-in-Koeln-Ehrenfeld.11021055.html', 'wg-zimmer-in-Koeln-Marienburg.10590012.html', 'wg-zimmer-in-Koeln-Nippes.11046954.html', 'wg-zimmer-in-Koeln-Koeln.9906542.html', 'wg-zimmer-in-Koeln-Altstadt-Sued.10954009.html', 'wg-zimmer-in-Koeln-Koeln.11074822.html', 'wg-zimmer-in-Koeln.11058466.html', 'wg-zimmer-in-Koeln-Altstadt-Sued.11074502.html', 'wg-zimmer-in-Koeln-Neustadt-Sued.10887214.html', 'wg-zimmer-in-Koeln-Altstadt-Sued.7751268.html', 'wg-zimmer-in-Koeln-Koeln.10984652.html', 'wg-zimmer-in-Koeln.10164365.html', 'wg-zimmer-in-Koeln-Lindenthal.9885436.html', 'wg-zimmer-in-Koeln-Junkersdorf.11074682.html']
20


302 => Captcha -->nochmal probieren
200 => Okay

wnen die Klasse auf einer Seite nicht mehr gefunden werden kann, enthält diese keine Angebote mehr

## Getting the Links

In [3]:
page = 0
failed_overview = 0
base_links=[]
campaign_links=[]

# main class for the links
target_class = "truncate_title noprint"

# class for the campaign links
campaign_class = "campaign_click"

last_page = page
cum_fails = 0
while page <= 20:
    # setting the url 
    url_overview_köln = "https://www.wg-gesucht.de/wg-zimmer-in-Koeln.73.0.1."+str(page)+".html"

    # Sends a GET request to the URL to retrieve its content; sometimes the Website tryes to redicrect the request to a captcha page, reloading solves the problem
    response = requests.get(url_overview_köln, allow_redirects=False)
    
    # test response code
    if response.status_code == 302 and failed_overview < 5:
        # try again
        failed_overview += 1 
        cum_fails += 1
        # if needed, add a sleep here
        continue
    elif failed_overview >= 5:
        print("Failed 5 times in a row")
        break
    
    page += 1
    failed_overview = 0

    # Parse results
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extracting all shared apartments
    elements = soup.find_all('h3', class_ = target_class)

    if len(elements) == 0:
        print("No more elements found")
        break

    
    for element in elements:
        for appartment in element.find_all('a', class_ = campaign_class):
            campaign_links.append(element.find('a').get('href'))
        for appartment in element.find_all('a', class_=''):
            base_links.append(element.find('a').get('href'))

    # just for debugging
    last_page = page -1


# TODO: aufhübschen und in Funktionen packen
print("Base links: ")
print(base_links)

set_base_links = set(base_links)
base_links = list(set_base_links)
print("Anzahl der Base links:" + str(len(base_links)))



print("Campaign links: " )
print(campaign_links)

set_campaign_links = set(campaign_links)
campaign_links = list(set_campaign_links)
print("Anzahl der Campaign links:" + str(len(campaign_links)))
print("Pages: "+str(last_page))
print("Failed: "+str(cum_fails))


KeyboardInterrupt: 

## Getting Information for different shared Appartments

nur verifizierte Unternehmen haben die Klasse "campaign clicks" --> sind auch unten als Werbung aufgeführt

verfifizierte Personen nicht



416 Ergebnisse sind okay. Erwartet hätten wir 420 -> ggf. Werbungen?

In [ ]:
# TODO: remove wrong links in the camoaign links (asset_id...)



In [4]:
# TODO: für eine Einzelseite die Attribute bestimmen
result_df = pd.DataFrame(columns=['Link', 'Title', 'Size', 'Rent', 'Extra Costs', 'Other Costs', 'Deposit', 'Redemption Agreement', 'Address', 'Available From', 'Available Till', 'Online Since'])

# Testing
test_counter = 0
failed_app_acc = 0

shared_app = 0

while shared_app < len(base_links):
    url_shared_app = "https://www.wg-gesucht.de" + str(base_links[shared_app]) 
    response = requests.get(url_shared_app, allow_redirects=False)
    print(response.status_code)
    # test response code
    if response.status_code == 302 and failed_app_acc < 5:
        # try again
        failed_app_acc += 1 
        cum_fails += 1
        # if needed, add a sleep here
        time.sleep(10)
        continue
    elif failed_app_acc >= 5:
        raise Exception("Failed 5 times in a row")

    if response.status_code == 503:
        print("Service Unavailable. Retrying in 15 seconds")
        time.sleep(15)
        failed_app_acc += 1 
        cum_fails += 1
        continue

    shared_app += 1
    failed_app_acc = 0

    soup = BeautifulSoup(response.text, 'html.parser')

    # Extracting the title
    title = soup.find('h1', class_='headline headline-detailed-view-title').text
    title = title.replace('\n', '').strip()

    # Extracting the size
    size = soup.find('b', class_='key_fact_value').text
    size = size.replace('\n', '').strip()

    # TODO: Adding the count of people in the flat
    # TODO: Maybe Restructure the replacing of the \n and strip to a function and do it in the df
    # Extracting the cost panel
    cost_panel = soup.find_all('div', class_='panel section_panel')[1]

    rent = cost_panel.find_all('span', class_='section_panel_value')[0].text
    rent = rent.replace('\n', '').strip()

    extra_costs = cost_panel.find_all('span', class_='section_panel_value')[1].text
    extra_costs = extra_costs.replace('\n', '').strip()

    other_costs = cost_panel.find_all('span', class_='section_panel_value')[2].text
    other_costs = other_costs.replace('\n', '').strip()

    deposit = cost_panel.find_all('span', class_='section_panel_value')[3].text
    deposit = deposit.replace('\n', '').strip()

    redemption_agreement = cost_panel.find_all('span', class_='section_panel_value')[4].text
    redemption_agreement = redemption_agreement.replace('\n', '').strip()



    # Address and Availability Panel
    address_panel = soup.find_all('div', class_='panel section_panel')[2]

    address = address_panel.find_all('span', class_='section_panel_detail')[0].text
    address = ' '.join(address.split())

    available_from = address_panel.find_all('span', class_='section_panel_value')[0].text
    available_from = available_from.replace('\n', '').strip()

    # Sometimes the available till is not available, so we need to check if the second span is 'frei bis:' --> if not we need to set it to n.a.
    if address_panel.find_all('span', class_='section_panel_detail')[2].text.strip() == 'frei bis:':
        available_till = address_panel.find_all('span', class_='section_panel_value')[1].text
        available_till = available_till.replace('\n', '').strip()
        online_since = address_panel.find_all('b', class_='noprint')[0].text
        online_since = online_since.replace('\n', '').strip()

    else:
        available_till = "n.a."
        online_since = address_panel.find_all('b', class_='noprint')[0].text
        online_since = online_since.replace('\n', '').strip()


    input_set = {'Link': url_shared_app, 'Title': title, 'Size': size, 'Rent': rent, 'Extra Costs': extra_costs, 'Other Costs': other_costs, 'Deposit': deposit, 'Redemption Agreement': redemption_agreement, 'Address': address, 'Available From': available_from, 'Available Till': available_till, 'Online Since': online_since}
    result_df = result_df._append(input_set, ignore_index=True)

    # Testing
    test_counter += 100
    if test_counter == 5:
        break 

result_df

200
200
200
200
200
200
200
200
200


KeyboardInterrupt: 

## Testing Area

In [ ]:
result_test_df = pd.DataFrame()
url_shared_app = "https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Agnesviertel.8024813.html" 
response = requests.get(url_shared_app, allow_redirects=False)
print(response.status_code)


soup = BeautifulSoup(response.text, 'html.parser')

# Extracting the title
title = soup.find('h1', class_='headline headline-detailed-view-title').text
title = title.replace('\n', '').strip()

# Extracting the size
size = soup.find('b', class_='key_fact_value').text
size = size.replace('\n', '').strip()




# TODO: Maybe Restructure the replacing of the \n and strip to a function and do it in the df
# Extracting the cost panel
cost_panel = soup.find_all('div', class_='panel section_panel')[1]

rent = cost_panel.find_all('span', class_='section_panel_value')[0].text
rent = rent.replace('\n', '').strip()

extra_costs = cost_panel.find_all('span', class_='section_panel_value')[1].text
extra_costs = extra_costs.replace('\n', '').strip()

other_costs = cost_panel.find_all('span', class_='section_panel_value')[2].text
other_costs = other_costs.replace('\n', '').strip()

deposit = cost_panel.find_all('span', class_='section_panel_value')[3].text
deposit = deposit.replace('\n', '').strip()

redemption_agreement = cost_panel.find_all('span', class_='section_panel_value')[4].text
redemption_agreement = redemption_agreement.replace('\n', '').strip()



# Address and Availability Panel
address_panel = soup.find_all('div', class_='panel section_panel')[2]

address = address_panel.find_all('span', class_='section_panel_detail')[0].text
address = ' '.join(address.split())

available_from = address_panel.find_all('span', class_='section_panel_value')[0].text
available_from = available_from.replace('\n', '').strip()

# Sometimes the available till is not available, so we need to check if the second span is 'frei bis:' --> if not we need to set it to n.a.
if address_panel.find_all('span', class_='section_panel_detail')[2].text.strip() == 'frei bis:':
    available_till = address_panel.find_all('span', class_='section_panel_value')[1].text
    available_till = available_till.replace('\n', '').strip()
    online_since = address_panel.find_all('b', class_='noprint')[0].text
    online_since = online_since.replace('\n', '').strip()

else:
    available_till = "n.a."
    online_since = address_panel.find_all('b', class_='noprint')[0].text
    online_since = online_since.replace('\n', '').strip()


input_set = {'Link': url_shared_app, 'Title': title, 'Size': size, 'Rent': rent, 'Extra Costs': extra_costs, 'Other Costs': other_costs, 'Deposit': deposit, 'Redemption Agreement': redemption_agreement, 'Address': address, 'Available From': available_from, 'Available Till': available_till, 'Online Since': online_since}
result_test_df = result_test_df._append(input_set, ignore_index=True)


print(address_panel.find_all('span', class_='section_panel_detail')[2].text.strip())
result_test_df


200
frei bis:


,Link,Title,Size,Rent,Extra Costs,Other Costs,Deposit,Redemption Agreement,Address,Available From,Available Till,Online Since
0,https://www.wg-gesucht.de/wg-zimmer-in-Koeln-A...,ab 1.9. Sonniges Zimmer/ Sunny room - tempor...,9m²,300€,100€,n.a.,400€,n.a.,Bernhard-Letterhaus-Str. 0 50670 Köln Agnesvie...,01.09.2024,28.02.2025,23.05.2024


Wer lebt in der WG:

In [ ]:
print(str(result_test_df['Link'][0]))


# TODO: Adding the count of people in the flat
anz_ges = soup.find('span', class_='mr5')
print(anz_ges['title'])

import re

def extract_wg_info(wg_string):
    # Regular expression to find the size of the WG (e.g., "3er WG")
    wg_size_pattern = re.compile(r'(\d+)er WG')
    wg_size_match = wg_size_pattern.search(wg_string)
    
    if wg_size_match:
        wg_size = int(wg_size_match.group(1))
    else:
        wg_size = None
    
    # Regular expression to find all groups (number and gender)
    pattern = re.compile(r'(\d+)([wmd])')
    
    # Find all matches in the input string
    matches = pattern.findall(wg_string)
    
    total_people = 0
    gender_counts = {'w': 0, 'm': 0, 'd': 0}
    
    for count, gender in matches:
        count = int(count)
        total_people += count
        if gender in gender_counts:
            gender_counts[gender] += count
    
    return wg_size, total_people, gender_counts

wg_string = anz_ges['title']
wg_size, total, genders = extract_wg_info(wg_string)
print(f"Größe der WG: {wg_size}")
print(f"Gesamtanzahl Personen: {total}")
print(f"Anzahl Frauen: {genders['w']}")
print(f"Anzahl Männer: {genders['m']}")
print(f"Anzahl Diverse: {genders['d']}")

https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Agnesviertel.8024813.html
3er WG (1w,1m,0d)
Größe der WG: 3
Gesamtanzahl Personen: 2
Anzahl Frauen: 1
Anzahl Männer: 1
Anzahl Diverse: 0


{'WG_Size': 3, 'Total_People': 2, 'w': 1, 'm': 1, 'd': 0}


WG Details

In [ ]:
detail_list = []
detail_dic = {}
details = soup.find_all('ul', class_='pl15 mb15')
for detail in details:
    spans = detail.find_all('span')
    detail_list.extend(spans)
details

for span in detail_list:
    span = span.text.replace('\n', '').strip()
    span = re.sub(r'\s+', ' ', span)
    
    if 'rauchen' in span.lower():
        detail_dic.update({'Rauchen': span}) 
    
    if 'alter' in span.lower():
        detail_dic.update({'Alter': span})
    
    if 'wohnungsgröße' in span.lower():
        detail_dic.update({'Wohnungsgröße': span})
    
    if 'sprache/n' in span.lower():
        detail_dic.update({'Sprache/n': span})
    
    if ('zweck-wg'  in span.lower()) or ('zweck wg' in span.lower()) or ('zweckgemeinschaft' in span.lower()):
        detail_dic.update({'Zweck-WG': 1})

    if ('frauen-wg' in span.lower()) or ('frauen wg' in span.lower()) or ('mädels wg' in span.lower()) or ('mädels-wg' in span.lower()):
        detail_dic.update({'Frauen-WG': 1})
    
    if ('männer-wg' in span.lower()) or ('männer wg' in span.lower()) or ('jungs wg' in span.lower()) or ('jungs-wg' in span.lower()):
        detail_dic.update({'Männer-WG': 1})
    
    if ('berufstätigen-wg' in span.lower()) or ('berufstätigen wg' in span.lower()) or ('berufstätige wg' in span.lower()) or ('berufstätige-wg' in span.lower()):
        detail_dic.update({'Berufstätigen-WG': 1})
    
    if ('gemischte wg' in span.lower()) or ('gemischte-wg' in span.lower()):
        detail_dic.update({'gemischte-WG': 1})

    if ('studenten-wg' in span.lower()) or ('studenten wg' in span.lower()) or ('studentinnen wg' in span.lower()) or ('studentinnen-wg' in span.lower()):
        detail_dic.update({'Studenten-WG': 1})
    
    if ('keine zweck-wg' in span.lower()) or ('keine zweck wg' in span.lower()) or ('keine zweckgemeinschaft' in span.lower()):
        detail_dic.update({'Zweck-WG': 0})
    
    if ('verbindung' in span.lower()):
        detail_dic.update({'Verbindung': 1})

    if('azubi-wg' in span.lower()) or ('azubi wg' in span.lower()) or ('azubis wg' in span.lower()) or ('azubis-wg' in span.lower()):
        detail_dic.update({'Azubi-WG': 1})
    
    if ('lgbtq' in span.lower()) or ('queer' in span.lower()):
        detail_dic.update({'LGBTQ': 1})

print(detail_dic)
    


{'Wohnungsgröße': 'Wohnungsgröße: 100m²', 'Alter': 'Bewohneralter: 29 bis 55 Jahre', 'Rauchen': 'Rauchen im Zimmer erlaubt', 'Zweck-WG': 1, 'Frauen-WG': 1, 'Berufstätigen-WG': 1, 'gemischte-WG': 1, 'Sprache/n': 'Sprache/n: Deutsch, Englisch, Spanisch, Französisch'}


Gesucht wird

In [ ]:
headline = soup.find('h4', class_='headline pb0', string = re.compile(r'Gesucht wird:'))

# Finde das nächste span-Element mit der Klasse "section_panel_detail" nach der Überschrift
if headline:
    section_detail = headline.find_next('span', class_='section_panel_detail')
    if section_detail:
        # Extrahiere den Text und entferne unnötige Leerzeichen
        text = section_detail.get_text(strip=True)
        print(text)
    else:
        print("Kein passendes span-Element gefunden.")
else:
    print("Kein passendes h4-Element gefunden.")

Geschlecht egal
                                        
                                        
                                        
                                        
                                                                                                                                    zwischen 24 und 60 Jahren


Angaben zum Objekt

In [ ]:
ang_z_objekt = soup.find('div', class_='utility_icons')
ang_z_objekt 

<div class="utility_icons">
<div class="text-center">
<span class="mdi mdi-lightbulb-on-outline mdi-42px round_gray_background" data-popover-content="#popover-energy-certification" data-toggle="popover"></span>
<div class="hidden noprint" id="popover-energy-certification">
<div aria-hidden="true" class="popover-close-me mdi mdi-close-circle-outline visible-xs-block visible-sm-block" onclick="hideinfo('popover-energy-certification');"></div>
                                        Ab dem 1. Mai 2014 ist die neue Energieeinsparverordnung (EnEV 2014) rechtswirksam. Immobilieninserate müssen seitdem zusätzliche Angaben zu energetischen Eigenschaften enthalten. Liegt noch kein Energieausweis vor, entfällt die Pflicht. Für denkmalgeschützte Häuser und Gebäude mit weniger als 50 Quadratmetern Nutzfläche ist kein Energieausweis nötig.
                                    </div>
<br><br/>

                                    
                                    
                                 

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

def extract_features_from_html(html_data):
    soup = BeautifulSoup(html_data, 'html.parser')
    containers = soup.find_all('div', class_='text-center')

    features = set()

    for container in containers:
        # Extrahiere den Text und bereinige ihn
        text = container.get_text(strip=True)
        # Splitte den Text in einzelne Features
        split_text = [t.strip() for t in text.split(',')]
        features.update(split_text)

    return features

def update_dataframe_with_features(html_data, df):
    # Extrahiere alle Features aus dem HTML
    new_features = extract_features_from_html(html_data)

    # Erstelle ein Dictionary für die neue Zeile
    new_row = {feature: 1 for feature in new_features}

    # Füge alle fehlenden Spalten hinzu und setze sie auf 0
    for feature in df.columns:
        if feature not in new_row:
            new_row[feature] = np.nan

    # Füge die neue Zeile zum DataFrame hinzu
    df = df._append(new_row, ignore_index=True)

    # Füge fehlende Spalten zu allen anderen Zeilen hinzu und setze sie auf 0
    for feature in new_row:
        if feature not in df.columns:
            df[feature] = np.nan
            df.loc[df.index != len(df) - 1, feature] = np.nan

    return df

# Beispielnutzung
url_shared_app = "https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Altstadt-Nord.11062804.html" 
response = requests.get(url_shared_app, allow_redirects=False)
soup = BeautifulSoup(response.text, 'html.parser')
html_data_2 = soup.find('div', class_='utility_icons')

url_shared_app = "https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Agnesviertel.8024813.html" 
response = requests.get(url_shared_app, allow_redirects=False)
soup = BeautifulSoup(response.text, 'html.parser')
html_data_1 = soup.find('div', class_='utility_icons')
print(html_data_1)
# Vorhandener DataFrame mit einer Spalte "Teppich"
df = pd.DataFrame({'Teppich': [1]})

# Aktualisiere den DataFrame mit den neuen Features aus den ersten HTML-Daten
df = update_dataframe_with_features(str(html_data_1), df)

# Aktualisiere den DataFrame mit den neuen Features aus den zweiten HTML-Daten
df = update_dataframe_with_features(str(html_data_2), df)

df.head()   



<div class="utility_icons">
<div class="text-center">
<span class="mdi mdi-lightbulb-on-outline mdi-42px round_gray_background" data-popover-content="#popover-energy-certification" data-toggle="popover"></span>
<div class="hidden noprint" id="popover-energy-certification">
<div aria-hidden="true" class="popover-close-me mdi mdi-close-circle-outline visible-xs-block visible-sm-block" onclick="hideinfo('popover-energy-certification');"></div>
                                        Ab dem 1. Mai 2014 ist die neue Energieeinsparverordnung (EnEV 2014) rechtswirksam. Immobilieninserate müssen seitdem zusätzliche Angaben zu energetischen Eigenschaften enthalten. Liegt noch kein Energieausweis vor, entfällt die Pflicht. Für denkmalgeschützte Häuser und Gebäude mit weniger als 50 Quadratmetern Nutzfläche ist kein Energieausweis nötig.
                                    </div>
<br><br/>

                                    
                                    
                                 

,Teppich,Altbau,4. OG,Dusche,Keller,Fahrradkeller,WLAN,2\n Minuten\n zu Fuß entfernt,Badewanne,entfällt die Pflicht. Für denkmalgeschützte Häuser und Gebäude mit weniger als 50 Quadratmetern Nutzfläche ist kein Energieausweis nötig.Baujahr 1928,teilmöbliert,Fernwärme,Ökostrom,Waschmaschine,schlechte Parkmöglichkeiten,Ab dem 1. Mai 2014 ist die neue Energieeinsparverordnung (EnEV 2014) rechtswirksam. Immobilieninserate müssen seitdem zusätzliche Angaben zu energetischen Eigenschaften enthalten. Liegt noch kein Energieausweis vor,möbliert,Laminat,Spülmaschine
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [ ]:
url_shared_app = "https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Nippes.3577779.html" 
response = requests.get(url_shared_app, allow_redirects=False)
soup3 = BeautifulSoup(response.text, 'html.parser')
html_data_3 = soup3.find('div', class_='utility_icons')

df = update_dataframe_with_features(str(html_data_3), df)

df.head()

,Teppich,Altbau,4. OG,Dusche,Keller,Fahrradkeller,WLAN,2\n Minuten\n zu Fuß entfernt,Badewanne,entfällt die Pflicht. Für denkmalgeschützte Häuser und Gebäude mit weniger als 50 Quadratmetern Nutzfläche ist kein Energieausweis nötig.Baujahr 1928,...,2. OG,Haustiere erlaubt,Gartenmitbenutzung,Flatrate,Gasheizung,gute Parkmöglichkeiten,Balkon,PVC,Aufzug,DSL
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,1.0,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
zimmer = soup.find('div', id='freitext_0', class_="wordWrap section_freetext").text.strip()
print('Zimmer: \n' + str(zimmer))

lage = soup.find('div', id='freitext_1', class_="wordWrap section_freetext display-none").text.strip()
print('Lage: \n' + str(lage))

wg_leben = soup.find('div', id='freitext_2', class_="wordWrap section_freetext display-none").text.strip()
print('WG-Leben: \n' + str(wg_leben))

sonstiges = soup.find('div', id='freitext_3', class_="wordWrap section_freetext")
print('Sonstiges: \n' + str(sonstiges))


Zimmer: 
AUF DEUTSCH: 
 Kleines, aber hübsches und sonniges Zimmerchen in schöner Wohnung (100qm) im Agnesviertel an freigeistigen, toleranten, verlässlichen Menschen zu vermieten.  
 Möblierung mit: Bett: 1.00 x 2.00, Nachttisch, offener Kleiderstange, Kommode, kleinem Schreibtisch, Stuhl.  
 Aufgrund der Zimmergröße bietet sich eher eine kurzfristige Nutzung an. Nicht geeignet für Vollzeitarbeit im Homeoffice. 
 InteressentIn sollte erstmal keine Anmeldung benötigen, sondern über eine andere Meldeadresse verfügen. 
 Falls es passt, ist eine Anmeldung  und auch eine langfristigere Vermietung möglich. 
 IN ENGLISH: 
 Small, but sunny, cosy and very quiet room in a nice flat in the beautiful Agnesviertel (Neustadt Nord) to rent to an open-minded, reliable,  relaxed person. The room is simply furbished with a bed, clothes rail, chest of drawers, desk.  
 Because of the small size of the room a short-time rental might be most suitable. A longer rental period might also be possible. 
 The 

Benötigte Unterlagen

In [ ]:
headline = soup.find('h3', class_='headline section_panel_title', string = re.compile(r'Benötigte Unterlagen'))

# Finde das nächste span-Element mit der Klasse "section_panel_detail" nach der Überschrift
if headline:
    test = True
    while test:
        ben_Unt = headline.find_next('b', class_='font-12px text-decoration-underline')
        if ben_Unt:
            # Extrahiere den Text und entferne unnötige Leerzeichen
            text = ben_Unt.get_text(strip=True)
            print(text)
            headline = ben_Unt
        else:
            test = False
    
else:
    print("Es werden keine Unterlagen benötigt.")

# TODO: itsmydata oder SCHUFA in

Bewerbermappe
Ausweis/ID


## Scraperapi-Approach

In [ ]:
config = configparser.ConfigParser()
config.read('local.ini')

# Extract the password
scraper_api_key = config['scraperapi']['key'].strip()
payload = {'api_key': scraper_api_key, 'url': "https://www.wg-gesucht.de/wg-zimmer-in-Koeln-Agnesviertel.8024813.html"}
r = requests.get('http://api.scraperapi.com', params=payload)
print(r.text)
print(r.status_code)


<!DOCTYPE html>
<html lang="de">
<head>
        <meta charset="utf-8"/>
    <meta name="viewport" content="width=device-width" id="viewport"/>
        <meta name="Author" content="WG-Gesucht.de"/>

            <meta name="ROBOTS" content="NOARCHIVE"/>
            
            <script data-cmp-ab="1">
            window.cmp_block_ignoredomains = [
                "stripe.com",
                "meineschufa.de",
                "my.matterport.com",
                "recaptcha.net",
                "paypal.com",
                "acrobatservices.adobe.com",
                "itsmydata.de",
                "static.elfsight.com"
            ];
        </script>
        <script type="text/javascript" data-cmp-ab="1" src="https://cdn.consentmanager.net/delivery/autoblock/15144.js" data-cmp-id="15144" data-cmp-host="a.delivery.consentmanager.net" data-cmp-cdn="cdn.consentmanager.net"></script>
    
    
                <script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
        new

# Creating Functions

In [ ]:
class Scraper():
    def __init__(self):
        self.scraper_api_key = self.set_api_param()
        self.non_spons_links = []

    def set_api_param(self):
        config = configparser.ConfigParser()
        config.read('local.ini')
        return config['scraperapi']['key'].strip()

    def get_page_via_api(self, url):
        payload = {'api_key': self.scraper_api_key, 'url': url}
        response = requests.get('http://api.scraperapi.com', params=payload)
        return response
    
    def get_page_direct(self, url):
        response = requests.get(url, allow_redirects=False)
        return response
    
    def collect_non_spons_links_from_response(self, response):
        soup = BeautifulSoup(response.text, 'html.parser')
        elements = soup.find_all('h3', class_ = "truncate_title noprint")
        for element in elements:
            # Filtering for the non sponsored appartments
            for appartment in element.find_all('a', class_=''):
                self.non_spons_links.append(element.find('a').get('href'))
        return self.non_spons_links
    

    

# Testing the Scraper() 
test = Scraper()
test_response = test.get_page_direct("https://www.wg-gesucht.de/wg-zimmer-in-Koeln.73.0.1.0.html")
print(test_response.status_code)
test.collect_non_spons_links_from_response(test_response)
print(test.non_spons_links)
print(len(test.non_spons_links))

# TODO: Error Handling for the direct request --> switchin to the api request
# TODO: save to json, after each page --> to avoid data loss
# TODO: Adding the sponsored appartments
# TODO: Loading the data of the appartments
# TODO: Add more attributes to the appartments
# TODO: Add the data to a dataframe
# TODO: write an output function to save the data
# TODO: better commenting
# TODO: new class for appartments


200
['/wg-zimmer-in-Koeln-Belgisches-Viertel.7798828.html', '/wg-zimmer-in-Koeln-Lindenthal.9499007.html', '/wg-zimmer-in-Koeln-Kalk.11019524.html', '/wg-zimmer-in-Koeln.11016563.html', '/wg-zimmer-in-Koeln-Altstadt-Sued.10663667.html', '/wg-zimmer-in-Koeln.8861077.html', '/wg-zimmer-in-Koeln-Marienburg.3216416.html', '/wg-zimmer-in-Koeln-Weidenpesch.8751447.html', '/wg-zimmer-in-Koeln-Neustadt-Sued.4864977.html', '/wg-zimmer-in-Koeln-Vingst.11016308.html', '/wg-zimmer-in-Koeln-Neustadt-Sued.7764003.html', '/wg-zimmer-in-Koeln-Humboldt-Gremberg.9090325.html', '/wg-zimmer-in-Koeln-Leverkusen-Wiesdorf.8836353.html', '/wg-zimmer-in-Koeln-Ehrenfeld.8183185.html', '/wg-zimmer-in-Koeln-Altstadt-Sued.7022036.html', '/wg-zimmer-in-Koeln-Neuehrenfeld-Neu-Ossendorf.11017025.html', '/wg-zimmer-in-Koeln-Altstadt-Sued.10081212.html', '/wg-zimmer-in-Koeln-Neustadt-Sued.9119691.html', '/wg-zimmer-in-Koeln-Neustadt-Sued.11019677.html', '/wg-zimmer-in-Koeln-Altstadt-Sued.11019572.html']


# Testing again